In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import yaml

In [ ]:
import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.family'] = 'Arial'

import scanpy as sc
# sc.settings.verbosity = 3
# sc.logging.print_versions()
Path("results/figures").mkdir(parents=True, exist_ok=True)
Path("results/data").mkdir(parents=True, exist_ok=True)
figure_type = 'svg'
sc.settings.figdir = "results/figures"
sc.settings.set_figure_params(fontsize=12, color_map='RdYlGn', dpi=80, dpi_save=1000)

import decoupler as dc
import celltypist as ct

In [ ]:
import sys
sys.path.extend(['../../mylibs'])

import scAnalysis_util

In [ ]:
sample_name = "ZT-288"
sample_path = Path("../../data/ZT-288/").absolute()
solo_out = sample_path / "starsolo_outputs/Solo.out/GeneFull/filtered"
solo_out_raw = sample_path / "starsolo_outputs/Solo.out/GeneFull/raw"
adata = sc.read_h5ad(solo_out / "matrix.stats.h5ad")
adata.X = adata.X.astype('float64')
adata.var_names = adata.var['gene_name'].apply(lambda x: x if x and str(x).strip() else None).fillna(adata.var['gene_ids'])
adata.var_names_make_unique()
adata

In [ ]:
#### Drop sum_umi_count(gene_id) == 0
count = adata.X.sum(axis=0)
count = np.array(count).flatten()
index = np.where(count>0)[0]
adata = adata[:, index].copy()

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)
adata

In [ ]:
#### Quality Control
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.lower().str.startswith((
    "mt-"
))
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.lower().str.startswith((
    "rps", "rpl"
))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.lower().str.contains('^hb[abgdez]$')

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True
)

In [ ]:
adata = adata[
    (adata.obs.n_genes_by_counts > 50) &
    (adata.obs.n_genes_by_counts < 4500) &
    (adata.obs.total_counts < 15000) &
    (adata.obs.pct_counts_mt < 8)
, :].copy()

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt", color="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="pct_counts_mt")

In [ ]:
#### Doublet detection
sc.pp.scrublet(adata)

In [ ]:
#### Normalization
adata.layers["counts"] = adata.X.copy()  # Store raw counts in a layer for highly variable genes
sc.pp.normalize_total(adata, target_sum=1e4)  # Normalizing to median total counts
sc.pp.log1p(adata)  # Logarithmize the data

In [ ]:
#### Highly Variable Genes
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=2000, layer="counts")
sc.pl.highly_variable_genes(adata)

In [ ]:
# Save raw expression values before variable gene subset, this will be used for regress_out and scale
adata.raw = adata

In [ ]:
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
sc.pp.scale(adata, zero_center=True, max_value=10)

In [ ]:
sc.tl.pca(adata, n_comps=50)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True, show=True)
sc.pl.pca(adata, color=["pct_counts_mt", "pct_counts_mt"], dimensions=[(0, 1), (2, 3)], ncols=2, size=2, show=True)

In [ ]:
adata_raw = adata.copy()

In [ ]:
adata = adata_raw.copy()
sc.pp.neighbors(adata, n_pcs=20, n_neighbors=15, metric="euclidean")
sc.tl.leiden(adata, resolution=0.8)
sc.tl.umap(adata, min_dist=0.8)
sc.pl.umap(adata, color=["leiden"])

In [ ]:
sc.tl.leiden(adata, flavor="igraph", key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(adata, flavor="igraph", key_added="leiden_res0_15", resolution=0.15)
sc.tl.leiden(adata, flavor="igraph", key_added="leiden_res0_2", resolution=0.2)
sc.tl.leiden(adata, flavor="igraph", key_added="leiden_res0_5", resolution=0.5)

In [ ]:
sc.pl.tsne(
    adata,
    color=["leiden_res0_1", "leiden_res0_15", "leiden_res0_2", "leiden_res0_5"],
    legend_loc="on data",
)

In [ ]:
adata_raw = adata.copy()

Manual

In [ ]:
adata = adata_raw.copy()
select_leiden = "leiden_res0_5"

In [ ]:
sc.tl.rank_genes_groups(adata, select_leiden, method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# Get top 10 highly expressed genes for each cluster in leiden
def get_top_genes_per_cluster(adata, groupby, n_top=10):
    # Get unique clusters
    clusters = adata.obs[groupby].unique()
    # Dictionary to store top genes for each cluster
    top_genes_dict = {}
    for cluster in clusters:
        # Get cells in this cluster
        cluster_mask = adata.obs[groupby] == cluster
        cluster_data = adata[cluster_mask, :]
        # Calculate mean expression for each gene in this cluster
        mean_expression = cluster_data.X.mean(axis=0)
        mean_expression = np.array(mean_expression).flatten()
        # Get indices of top expressed genes
        top_indices = np.argsort(mean_expression)[-n_top:][::-1]
        # Get gene names
        top_genes = adata.var_names[top_indices].tolist()
        top_genes_dict[cluster] = top_genes
    return top_genes_dict

# Get top 10 genes for each leiden_res0_2 cluster
top_genes_per_cluster = get_top_genes_per_cluster(adata, groupby=select_leiden, n_top=10)

# Display results
for cluster, genes in sorted(top_genes_per_cluster.items()):
    print(f"Cluster {cluster}: {', '.join(genes)}")

In [ ]:
top_genes_per_cluster = dict(sorted(top_genes_per_cluster.items(), key=lambda x: int(x[0])))
sc.pl.dotplot(adata, top_genes_per_cluster, groupby=select_leiden)

In [ ]:
sc.pl.violin(adata, ["PID1", "IDUA"], groupby=select_leiden)

In [ ]:
sc.pl.tsne(adata, color=['FMNL2'], cmap='RdBu_r', vcenter=0)